In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import dvu
from copy import deepcopy
import sys
from numpy import ceil
sys.path.append('../notebooks')
from tqdm import tqdm
from neuro import config
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
import neuro.viz
from PIL import Image
from neuro.flatmaps_helper import load_flatmaps, load_custom_rois
from neuro import analyze_helper

In [44]:
res = defaultdict(list)
for model in ['llama_model', 'opt_model']:
    for subject in ['S01', 'S02', 'S03']:
        if not os.path.exists(join(config.SAVE_DIR_FMRI, model, 'voxel_performances', f'UT{subject}_voxel_performance.jbl')):
            print(f"Voxel performance file for {subj} does not exist. Please run the analysis first.")
            sys.exit(1)
        voxel_perfs = joblib.load(join(config.SAVE_DIR_FMRI, model, 'voxel_performances', f'UT{subject}_voxel_performance.jbl'))
        if model == 'llama_model':
            voxel_perfs = voxel_perfs[0]
        res['subject'].append(subject)
        res['corrs_test_mean'].append(voxel_perfs.mean())

        # neuro.viz.quickshow(voxel_perfs, vmin=-0.4, vmax=0.4, fname_save=f'wordrate_baseline/{subject}_{model}.png')
        res['model'].append(model)

In [ ]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/jul15_2025_wordrate_baseline'
rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)

In [ ]:
for subject in rr['subject'].unique():
    neuro.viz.quickshow(
        rr[rr['subject'] == subject]['corrs_test'].values[0],
        subject=subject,
        fname_save=f'wordrate_baseline/{subject}_wordrate.png',
        vmin=-0.4, vmax=0.4,
    )

In [50]:
res = pd.DataFrame(res)
res_wr = rr[['subject', 'corrs_test_mean']]
res_wr['model'] = 'word-rate'
res = pd.concat([res, res_wr], ignore_index=True)

/tmp/ipykernel_2679456/801879030.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_wr['model'] = 'word-rate'


In [63]:
print(
    res.pivot(index='model', columns='subject', values='corrs_test_mean')
    .to_latex(float_format='%.2f')
)

\begin{tabular}{lrrr}
\toprule
subject & S01 & S02 & S03 \\
model &  &  &  \\
\midrule
llama_model & 0.08 & 0.15 & 0.17 \\
opt_model & 0.08 & 0.14 & 0.16 \\
word-rate & 0.04 & 0.03 & 0.03 \\
\bottomrule
\end{tabular}

